# Installation Requirements

*   [Neo4j](https://neo4j.com/) is used as the underlying graph vector database and retriever for GraphRag.
*   Langchain is used to orchestrate the database (retriever) and LLM for final output.
*   This notebook aims to build the RAG system by orchestrating the retriever (neo4j) and LLM (Llama 3.2 is used in this notebook).

In [1]:
!pip install neo4j
!pip install langchain
!pip install langchain_community
!pip install transformers torch
!pip install gradio
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.

# Setting up vector database

In [2]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [3]:
#Parameters to connect Neo4j graph database
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
AUTH = (NEO4J_DATABASE, NEO4J_PASSWORD)


with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

# Langchain

Two main tasks here are to define class NorEmbeddings and RetrievalQAWithBlogChain.


*   NorEmbeddings: used to populate embeddings (if not exists) when Initializing a Neo4jVector instance
*   RetrievalQAWithBlogChain: The strategy used in this chain first retrieves the most relevant chunk based on embedding similarity, then combines all connected chunks from the same blog, and finally inputs the combined content into the LLM for generating the final output.



In [5]:
!huggingface-cli login --token #add your tokens here

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `coding` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `coding`


In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Load the open-source model from Hugging Face
#Norwegian is not among the languages Llama 3.2 officially supports (English, German, French, Italian, Portuguese, Hindi, Spanish, and Thai)
#It works not bad on this use case, but other models finetuned for Norwegian can also be considered
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # EleutherAI/gpt-neo-1.3B
gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(model_name)
gen_model.to('cuda')

# Create a HuggingFace pipeline
hf_pipeline = pipeline("text-generation", model=gen_model, tokenizer=gen_tokenizer,max_new_tokens = 2048, truncation=True)

# Wrap the HuggingFace pipeline in a LangChain LLM wrapper
llm = HuggingFacePipeline(pipeline=hf_pipeline, verbose=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
<ipython-input-6-e44c1ecff1df>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline, verbose=True)


In [7]:
from typing import List
from langchain.embeddings.base import Embeddings
from transformers import AutoModel, AutoTokenizer
import torch

#Embedding Class to be used for text embedding
class NorEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        # Tokenize and create embeddings for documents
        inputs = self.tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state
        # Mean pooling to get a single vector per document
        pooled_embeddings = embeddings.mean(dim=1).numpy()
        return pooled_embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        # Tokenize and create embeddings for the query
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state
        # Mean pooling to get a single vector for the query
        pooled_embedding = embeddings.mean(dim=1).numpy()
        return pooled_embedding.flatten().tolist()


In [8]:
from typing import Any, Dict, List, Optional, Tuple
import re

import inspect
from langchain_core.callbacks import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from pydantic import Field

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.qa_with_sources.base import BaseQAWithSourcesChain

#chain to retrieve contents based on sementic search, refine the contents using knowledge graph and then input LLM for summarizing
class RetrievalQAWithBlogChain(BaseQAWithSourcesChain):
    """Question-answering with sources over an index."""

    retriever: BaseRetriever = Field(exclude=True)
    """Index to connect to."""
    reduce_k_below_max_tokens: bool = False
    """Reduce the number of results to return from store based on tokens limit"""
    max_tokens_limit: int = 3375
    """Restrict the docs to return from store based on tokens,
    enforced only for StuffDocumentChain and if reduce_k_below_max_tokens is to true"""
    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._get_docs).parameters
        )
        if accepts_run_manager:
            docs = self._get_docs(inputs, run_manager=_run_manager)
        else:
            docs = self._get_docs(inputs)  # type: ignore[call-arg]

        answer = self.combine_documents_chain.run(
            input_documents=docs, callbacks=_run_manager.get_child(), **inputs
        )
        answer, sources = self._split_sources(answer)
        result: Dict[str, Any] = {
            self.answer_key: answer,
            self.sources_answer_key: sources,
        }
        if self.return_source_documents:
            result["source_documents"] = docs
        return result

    def _split_sources(self, answer: str) -> Tuple[str, str]:
        """Split sources from answer."""
        # Regular expression patterns
        source_pattern = r'Source:\s*(https?://\S+)'  # To match the Source link
        raganswer_pattern = r'RagAnswer:\s*(.*)'     # To match everything after RagAnswer

        # Find the source link and RagAnswer using the regular expressions
        source_match = re.search(source_pattern, answer)
        raganswer_match = re.search(raganswer_pattern, answer, re.DOTALL)

        # Extract the matched content
        sources = source_match.group(1) if source_match else None
        answer = raganswer_match.group(1).strip() if raganswer_match else None

        return answer, sources

    def _reduce_tokens_below_limit(self, docs: List[Document]) -> List[Document]:
        num_docs = len(docs)

        if self.reduce_k_below_max_tokens and isinstance(
            self.combine_documents_chain, StuffDocumentsChain
        ):
            tokens = [
                self.combine_documents_chain.llm_chain._get_num_tokens(doc.page_content)
                for doc in docs
            ]
            token_count = sum(tokens[:num_docs])
            while token_count > self.max_tokens_limit:
                num_docs -= 1
                token_count -= tokens[num_docs]

        return docs[:num_docs]

    def _get_docs(
        self, inputs: Dict[str, Any], *, run_manager: CallbackManagerForChainRun
    ) -> List[Document]:
        question = inputs[self.question_key]
        docs = self.retriever.invoke(
            question, config={"callbacks": run_manager.get_child()}
        )
        # print('docs before refine')
        # print(docs)
        chunkid = docs[0].metadata['chunkId']
        blogid = docs[0].metadata['formId']
        # print('blog id is: ', blogid)
        #get all the chunks belonging to the same blog
        refine_query = (
                # f"MATCH (chunk:Chunk) WHERE chunk.chunkId = {chunkid}" +
                f"MATCH (allchunks:Chunk)-[PART_OF]->(blog:Blog) WHERE blog.fileId ='{blogid}' " +
                f"RETURN reduce(str='', k IN {TEXT_NODE_PROPERTIES} |"
                " str + '\\n' + k + ': ' + coalesce(allchunks[k], '')) AS page_content, "
                "allchunks {.*, `"
                + VECTOR_EMBEDDING_PROPERTY
                + "`: Null, "
                + ", ".join([f"`{prop}`: Null" for prop in TEXT_NODE_PROPERTIES])
                + "} AS metadata "
                +"ORDER BY allchunks.chunkSeqId ASC"
            )
        # print(refine_query)

        # docs = self.retriever.invoke(
        #     refine_query, config={"callbacks": run_manager.get_child()}
        # )
        docs = []
        docs_results = self.retriever.vectorstore.query(refine_query)
        for doc in docs_results:
            docs.append(Document(page_content=doc['page_content'].replace('\xa0',""), metadata=doc['metadata']))
        # print('docs after refine')
        # print(docs)
        return self._reduce_tokens_below_limit(docs)

    async def _aget_docs(
        self, inputs: Dict[str, Any], *, run_manager: AsyncCallbackManagerForChainRun
    ) -> List[Document]:
        question = inputs[self.question_key]
        docs = await self.retriever.ainvoke(
            question, config={"callbacks": run_manager.get_child()}
        )
        chunkid = docs[0].metadata['chunkId']
        blogid = docs[0].metadata['formId']
        #get all the chunks belonging to the same blog
        refine_query = (
                # f"MATCH (chunk:Chunk) WHERE chunk.chunkId = {chunkid}" +
                f"MATCH (allchunks:Chunk)-[PART_OF]->(blog:Blog) WHERE blog.fileId ='{blogid}' " +
                f"RETURN reduce(str='', k IN {TEXT_NODE_PROPERTIES} |"
                " str + '\\n' + k + ': ' + coalesce(allchunks[k], '')) AS page_content, "
                "allchunks {.*, `"
                + VECTOR_EMBEDDING_PROPERTY
                + "`: Null, "
                + ", ".join([f"`{prop}`: Null" for prop in TEXT_NODE_PROPERTIES])
                + "} AS metadata "
                +"ORDER BY allchunks.chunkSeqId ASC"
            )
        # print(refine_query)
        docs = self.retriever.ainvoke(
            refine_query, config={"callbacks": run_manager.get_child()}
        )
        return self._reduce_tokens_below_limit(docs)

    @property
    def _chain_type(self) -> str:
        """Return the chain type."""
        return "retrieval_qa_with_blogs_chain"



In [9]:
VECTOR_NODE_LABEL = 'Chunk'
TEXT_NODE_PROPERTIES = ['text']
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'
VECTOR_INDEX_NAME = 'vector_chunks'

#this retrieval query defines what to return
#if not given, all the TEXT_NODE_PROPERTIES will be returned as text, and other properties except embedding, text, and id will be returned as metadata
retrieval_query = (
                "ORDER BY score DESC" +
                "LIMIT 1" +
                f"RETURN reduce(str='', k IN {TEXT_NODE_PROPERTIES} |"
                " str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, "
                "node {.*, `"
                + VECTOR_EMBEDDING_PROPERTY
                + "`: Null, id: Null, "
                + ", ".join([f"`{prop}`: Null" for prop in TEXT_NODE_PROPERTIES])
                + "} AS metadata, score"

            )
#Initialize and return a Neo4jVector instance from an existing graph.
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=NorEmbeddings("NbAiLab/nb-bert-base"),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=TEXT_NODE_PROPERTIES,
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
    # retrieval_query=retrieval_query
)
retriever = neo4j_vector_store.as_retriever(search_kwargs={"k": 1})

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [10]:
from langchain.prompts import PromptTemplate

# Create a custom prompt template
template = """Suppose that you are an assistant for an IT consulting company. Given the following extracted parts of a long document and a question, create a final answer.
Don't repeat your answer.
spørsmålet: {question}
=========
{summaries}
=========
RagAnswer:"""

# template = """Tenk deg at du er assistent for et IT-konsulentselskap. Basert på følgende innhold, svar konsist på spørsmålet, og ikke gjenta svaret ditt:
# spørsmålet: {question}
# =========
# {summaries}
# =========
# RagAnswer:"""
custom_prompt = PromptTemplate(
    template=template,
    input_variables=["summaries", "question"],
)

chain = RetrievalQAWithBlogChain.from_chain_type(
    #combine_documents_chain=combine_prompt_template,
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    verbose=False,  # Enables detailed logging of inputs and outputs,
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)

In [11]:
# query = "Hva skal jeg gjøre når jeg møter hinderinger som prosjektleder?"
# result = chain.invoke(chain.prep_inputs(query),
#         return_only_outputs=True,)

# print("Answer:", result['answer'])
# print("Sources:", result['sources'])

#UI

## One Dialog Window

## Multi-Dialog Window

In [12]:
import gradio as gr
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Neo4jVector
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Step 1: Create the function for Gradio interface to process user input
def generate_answer(question):
    result = chain.invoke(chain.prep_inputs(question),
        return_only_outputs=True,)
    answer, sources = result['answer'], result['sources']
    return f"Svaret:\n {answer}\n\nKilder:\n{sources}"

# Step 2: Set up the Gradio interface
gr_interface = gr.Interface(
    fn=generate_answer,
    inputs="text",
    outputs="text",
    title="Kantega Assistent",
    description="Still et spørsmål og få svar med kilder"
)

# Step 3: Launch the Gradio interface and get the public URL
gr_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c631de115c2274d7cc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
import gradio as gr
from langchain.chains import RetrievalQA
from langchain.vectorstores import Neo4jVector  # Replace with your specific vector store
from transformers import pipeline


# Function to process user input and use RAG to generate a response
def rag_chatbot(input_text, history):
    # Append user's input to conversation history
    history.append(("User", input_text))
    # Use the RAG system to generate the response
    rag_response = chain.invoke(chain.prep_inputs(input_text),
        return_only_outputs=True)
    response = f"{rag_response['answer']}\n\nKilder:\n{rag_response['sources']}"

    # Append the chatbot's RAG response to conversation history
    history.append(("Chatbot", response))

    # Display updated conversation history
    return history, history, ""

# Gradio Interface for the chatbot with RAG system
with gr.Blocks() as demo:
    chatbox = gr.Chatbot()
    user_input = gr.Textbox(placeholder="Type your message here...", label="Input")
    conversation_state = gr.State([])  # Keep track of conversation history

    # Update the chatbox with the RAG-generated response when the user sends a message
    user_input.submit(rag_chatbot, inputs=[user_input, conversation_state], outputs=[chatbox, conversation_state, user_input])

# Launch the Gradio interface
demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://23e98d71a57fb1865b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
